# Variational Quantum Eigensolver

## Summary
* Aims to find minimize the Hamiltonian, i.e. find the ground state energy of the system.
* Often used in Quantum Chemistry to find the ground state energy of a molecule

#### Method
1. Define the Hamiltonian $\hat{H}$ of the system and choose an ansatz with parameters $\overrightarrow{\theta}$.
2. Apply the ansatz to prepare the trial quantum state $\ket{\psi(\overrightarrow{\theta})}$.
3. Measure the expectation value $E(\overrightarrow{\theta}) = \bra{\psi(\overrightarrow{\theta})} \hat{H} \ket{\psi(\overrightarrow{\theta})}$
4. Optimization: Use a classical optimizer such as Gradient Descent to update $\overrightarrow{\theta}$ such that it minimizes $E(\overrightarrow{\theta})$. The resulting state should be $\ket{\Psi_0}$, which will give us $E_0$.

#### Considerations
* particularly suitable for near-term quantum devices because it requires relatively short quantum circuits and can tolerate certain level of noise.
* However, the ansatz needs to be expressive enough to capture the true ground state, while being simple enough to allow trainability and execution on current quantum hardware.
---

In [1]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.data.data_manager import params
import matplotlib.pyplot as plt

### Codercise V.2.1

In [2]:
dev = qml.device("default.qubit", wires=4)

@qml.qnode(dev)
def strongly_entangling_ansatz(observable, params):
    """Applies an ansatz with moderate entanglement.

    Args:
        observable (qml.op): a pennylane operator whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz. They have the shape of `qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=n_bits)`

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the expectation value of the given observable.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    for i in range(n_bits):
        qml.Rot(params[0][i][0],params[0][i][1],params[0][i][2],wires=i)
    qml.CNOT(wires=[0, n_bits-1])
    for i in range(1, n_bits):
        qml.CNOT(wires=[i, i-1])

    return qml.expval(observable)

def cost_function(observable, observableparams):
    """Computes the cost function we want to minimize.

    Args:
        observable (qml.Hamiltonian): a pennylane Hamiltonian whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz.

    Returns:
        (np.tensor): a numpy tensor of 1 element corresponding to the cost function value
    """
    ##################
    # YOUR CODE HERE #
    ##################
    return strongly_entangling_ansatz(observable, observableparams)
    
def optimizer(observable, params, learning_rate=0.1, steps=100):
    """Updates the parameters to minimize the cost function value.

    Args:
        observable (qml.Hamiltonian): a pennylane Hamiltonian whose expectation value we want to measure.
        params(np.array): an array with the trainable parameters of the ansatz.

    Returns:
        (np.array): an array with the optimized trainable parameters.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    optimizer = qml.GradientDescentOptimizer(learning_rate)
    optimized_params = params.copy()
    cost = lambda p: cost_function(observable, p)

    for _ in range(steps):
        optimized_params = optimizer.step(cost, optimized_params)

    return optimized_params
    

In [3]:
n_bits = 2
dev = qml.device("default.qubit", wires=n_bits, shots=None)
np.random.seed(41)
def build_hamiltonian():
    """Build the Hamiltonian.

    Returns:
        (qml.Hamiltonian): Hamiltonian operator.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    return qml.dot([1, -1], [qml.Z(0) @ qml.Z(1), qml.X(0) @ qml.X(1)])

def run_vqe():
    """Run VQE algorithm with initial parameters defined by the user.

    Returns:
        array(float): Ground state energy of the Hamiltonian.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    # initialise parameters
    shape = qml.StronglyEntanglingLayers.shape(n_layers=1, n_wires=n_bits)
    initial_params = np.random.randn(*shape, requires_grad=True)

    H = build_hamiltonian()
    opt_params = optimizer(H, initial_params)
    
    return cost_function(H, opt_params)

 

In [4]:
e0 = run_vqe()
print("Ground state energy:", e0)

Ground state energy: -1.999999036614355


### Codercise V.2.2.a

In [6]:
# Couldn't get this one to work!

from pennylane import qchem

# Import the H2 molecule dataset
dataset = qml.data.load('qchem', folder_path="/tmp", molname="H2")[0]
# Define Hamiltonian and qubits
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)
# The Hartree-Fock State
hf = dataset.hf_state
# Define the single and double excitations
singles, doubles = qchem.excitations(electrons=2, orbitals=qubits)
num_params = len(singles) + len(doubles)

def hf_ansatz(params):
    """Build the Hartree-Fock ansatz.
    
    Args:
        params (np.array): An array with the angles of the single and double excitations.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    qml.AllSinglesDoubles(params, range(qubits), hf)

dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev, interface="autograd")
def cost_hf(params):
    """Build the cost function using the Hartree-Fock ansatz.
    Args:
        params (np.array): An array with the angles of the single and double excitations.

    Returns:
        (np.tensor): Energy of the Hamiltonian.
    """
    ##################
    # YOUR CODE HERE #
    ##################
    hf_ansatz(params)
    return qml.expval(H)



FileNotFoundError: https://cloud.pennylane.ai/datasets/v2/download/h2-molecule-h2-sto-3g-0.742-n%5Ca-4

### Codercise V.2.2.b